# checking individual genes

In [1]:
import helpers
import numpy as np
import pandas as pd

In [2]:
df_sc_rnaseq, df_sc_metadata = helpers.datasets.load_jerby_arnon_hg19_tpm()

In [3]:
import upath

In [5]:
def compute_gene_ratios(df_sc_rnaseq: pd.DataFrame, df_sc_metadata: pd.DataFrame):
    df = df_sc_metadata
    # drop cells with missing cell_type
    df = df.dropna(subset=["cell_type"])
    # drop cells with cell_type "CAF" or "Endothelial"
    df = df[~df["cell_type"].isin(["CAF", "Endothelial"])]
    # add a string column with value "malignant" if cell type is "Malignant" or "other" otherwise
    df = df.assign(cell_group=df["cell_type"].map(lambda x: "malignant" if x == "Malignant" else "other"))

    # add gene expression for each single_cell_id
    df = df[["cell_group"]].merge(df_sc_rnaseq.T, left_index=True, right_index=True)
    df = df.groupby("cell_group").mean().T
    df = df.assign(malignant_ratio=df["malignant"] / df["other"])
    df = df.sort_values("malignant_ratio", ascending=False)
    # .join(df_sc_rnaseq.stack(), on="single_cell_id", how="left", validate="many_to_one")

    return df


compute_gene_ratios(
    df_sc_rnaseq=df_sc_rnaseq,
    df_sc_metadata=df_sc_metadata,
)

cell_group,malignant,other,malignant_ratio
MIR3186,0.001562,0.0,inf
MIR4789,0.001308,0.0,inf
C5orf46,0.002975,0.0,inf
TMEM114,0.001075,0.0,inf
MAGEA9B,0.002778,0.0,inf
...,...,...,...
SPANXN1,0.000000,0.0,NaN
SPANXN5,0.000000,0.0,NaN
SPINK14,0.000000,0.0,NaN
TMEM88B,0.000000,0.0,NaN


In [ ]:
def make_scatter_plot_of_gene_false_positive_count_and_malignant_ratio(
    df_gene_stats: pd.DataFrame, df_gene_malignant_ratio: pd.Series
):
    pass

In [ ]:
!gsutil ls gs://liulab/

In [ ]:
# compute ratio of mean expression of each gene in malignant cells vs other cells
df_jerby_arnon["ratio"] = df_jerby_arnon["malignant"] / df_jerby_arnon["other"]

In [ ]:
# load jerby-arnon scrna-seq